# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

# Домашнее задание

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [ ]:
pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 66.4 MB/s eta 0:00:00


In [11]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares

from metrics import precision_at_k, recall_at_k
from recommenders import MainRecommender
from utils import prefilter_items, get_result_table

import warnings
warnings.filterwarnings('ignore')

In [12]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'}, inplace=True)

In [13]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [14]:
data_train = data[data['week_no'] < data['week_no'].max() - 3]
data_test = data[data['week_no'] >= data['week_no'].max() - 3]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631.0,1.0,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631.0,1.0,0.0,0.0


Собираем items из тестовой выборки, сохраняем их в датафрейм result (функция для этого лежит в metrics):

In [15]:
result = get_result_table(data_test)
result.head(2)

,user_id,actual
0,1,"[837208, 849264, 851231, 856942, 861272, 86474..."
1,2,"[868389, 868547, 883665, 911974, 925862, 93493..."


С помощью функции prefilter_items из metrics снижаем количество элементов:

In [16]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print(f'Количество элементов уменьшено с {n_items_before} до {n_items_after}')

Количество элементов уменьшено с 62251 до 5001


In [17]:
main_data_train = MainRecommender(data_train)
user_item_matrix = main_data_train.user_item_matrix
user_item_matrix.head(3)

item_id,42346,42521,78986,108646,117847,244960,818981,819048,819255,819308,...,12946257,12949590,12949855,13002975,13003092,13003094,13071902,13072793,13158064,13158992
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
id_to_itemid, id_to_userid, itemid_to_id, userid_to_id =\
    main_data_train.id_to_itemid, main_data_train.id_to_userid, main_data_train.itemid_to_id, main_data_train.userid_to_id

Словарь с рекомендациями для всех пользователей:

In [27]:
%%time
user_all = MainRecommender(data_train, als_recommend=True)
user_all.generate_recommendations()

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 11min 54s, sys: 7min 27s, total: 19min 22s
Wall time: 13min 7s


Из полученных рекомендация для всех посмотрим рекомендации для одного пользователя №215:

In [28]:
recs_215 = user_all.recommendations[215]

In [29]:
recs_215

[854852, 948650, 863447, 1070702, 1135476]